In [1]:
import pandas as pd
import urllib.request
import os
from datetime import datetime,date
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import json
import yaml
from calendar import monthrange
import xarray as xr
import cdsapi
import zipfile
from glob import glob
#important
#before running this script, please git pull, then git checkout c3sprod


# wishlist
# - put land hydrology just below cryosphere
# - update repository and change to c3sprod
#   - git checkout c3sprod
#   - git pull
# - download datasets that only contain info in the time dimension from CDS
# - try to make the code more path-independent
# - install on athos and run every week. update some figure repo



def extract_dates_from_TSI():
    url='https://gerb.oma.be/tsi/C3S_RMIB_daily_TSI_composite_ICDR_v3_latest.txt'
    c=pd.read_csv(url,skiprows=128,delim_whitespace=True,header=None)
    return pd.Timestamp(str(c[3].iloc[0])),pd.Timestamp(str(c[3].iloc[-1]))
def extract_dates_icesheets(datasets_dir):
    # opens TCDR and ICDR files for both Antarctica and Greenland, then computes max/min dates
    # needs adjustment for per product sorting
    fnames = glob(datasets_dir+'C3S_*IS_RA*.nc')
    datebegs = []
    dateends = []
    for fname in fnames:
        nc = xr.open_dataset(fname)
        datebegs.append(pd.Timestamp(nc['time'].values[0]))
        dateends.append(pd.Timestamp(nc['time'].values[-1]))
    return min(datebegs),max(dateends)
def extract_dates_massbalance(datasets_dir):
    fname = datasets_dir+'C3S_GMB_GRACE_vers4.nc'
    nc = xr.open_dataset(fname)
    return pd.Timestamp(nc['time'].values[0]),pd.Timestamp(nc['time'].values[-1])
def extract_dates_derived_glaciers(jfile):
    with open(jfile) as f:
        gen= yaml.safe_load(f)
    years = gen['labels']['hydrological_year'].keys()
    ymin=int(min(years))
    ymax=int(max(years))+1
    mmax=9
    dmax=30
    mmin=4
    dmin=1
    print('Glaciers ',ymin,ymax)
    return pd.Timestamp(f'{ymin}-{mmin}-{dmin}'), pd.Timestamp(f'{ymax}-{mmax}-{dmax}')
def extract_dates_lake_levels(datasets_dir):
    # need better way of estimating this
    # for v4, according to overview pages, coverage goes until 2022
    # need to implement download of all lakes data and check their time coverage
    print('...for lake water levels, only reading info for Lake Volta...')
    fname = datasets_dir+'C3S_LWL_N-AFRICA_VOLTA_altimetry_4.0_19921013_20221222_R20230126.nc'
    nc = xr.open_dataset(fname)
    return pd.Timestamp(nc['time'].values[0]),pd.Timestamp(nc['time'].values[-1])
def datemax2(row):
    row = row.dropna()
    ymax = int(max(row['year']))
    if 'month' in row.keys():
        mmax = int(max(row['month']))
    else:
        mmax=12
    xx,dmax=monthrange(ymax,mmax)
    datemax = pd.Timestamp(f'{ymax}-{mmax}-{dmax}')
    return datemax
def datemin2(row):
    row = row.dropna()
    ymin = int(min(row['year']))
    if 'month' in row.keys():
        mmin = int(min(row['month']))
    else:
        mmin=1
    xx,dmin=monthrange(ymin,mmin)
    datemin = pd.Timestamp(f'{ymin}-{mmin}-{dmin}')
    return datemin
def extract_dates_wv(jfilepath):
    f = open(jfilepath)
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
    # print(data)
    df = pd.DataFrame(data)
    df['datemax'] = df.apply(datemax2,axis=1)
    df['datemin'] = df.apply(datemin2,axis=1)
    datemin = df['datemin'].min()
    datemax = df['datemax'].max()
    return datemin,datemax 
def check_time_agg(row):
    row = row.dropna()
    if 'time_aggregation' in row.keys():
        time_aggregation = row['time_aggregation'][0]
        # print('time_agg',time_aggregation)
        if time_aggregation in ['daily_average','daily_mean','day','day_average']: 
            time_agg = 'day'
            return time_agg
        elif time_aggregation == 'daily':
            time_agg='daily'
            return time_agg
        #note interim solution for 5-daily-composite...
        elif time_aggregation in [
            'monthly_average',
            '5_daily_composite',
            'monthly_mean',
            '27_days',
            'month',
            'monthly',
            'month_average',
            '10_day_average', # debatable..
            ]:
            time_agg = 'monthly'
            return time_agg
        else:
            # print(row)
            print('Could not determine time_agg')
            raise SystemExit
    elif 'period' in row.keys(): # applies to ice_sheets
        time_agg = 'period'
        return time_agg
    elif 'temporal_aggregation' in row.keys():
        time_aggregation = row['temporal_aggregation'][0]
        if time_aggregation in ['monthly','6-hourly']:
            time_agg='monthly'
        elif time_aggregation == 'daily':
            time_agg='daily'
        else:
            print('Error in temporal aggregation')
            raise SystemExit
        return time_agg
    else:
        if ('day' in row.keys()):
            time_agg='day'
            return time_agg
        elif  'nominal_day' in row.keys():
            time_agg='nominal_day'
            return time_agg
        else:
            time_agg = 'monthly'
            return time_agg
def compute_datemax(row):
 
    time_agg=check_time_agg(row) # check time aggregation of data in this row
    # print('time_agg',time_agg)
    if time_agg =='period':
        per_str=max(row['period'])
        ymax=int(per_str[5::])
        mmax=9
        dmax=30
    else: 
        ymax = int(max(row['year']))
        if 'month' in row.keys():
            mmax = int(max(row['month']))
        else:
            mmax=12
        xx,ndays = monthrange(ymax,mmax)
        if time_agg in ['day','nominal_day']:
            # print(row[time_agg])
            dmax = int(max(row[time_agg])) 
        elif time_agg in ['daily']:
            dmax = int(max(row['day'])) 
        else:
            dmax = ndays # last day of month
        if dmax >ndays:
            print('Beware error in allowed dates...')
            # print(row)
            dmax=ndays

    datemax = pd.Timestamp(f'{ymax}-{mmax}-{dmax}')
    return datemax
def compute_datemin(row):
    time_agg=check_time_agg(row) # check time aggregation of data in this row
    if time_agg =='period':
        per_str=max(row['period'])
        ymin=int(per_str[0:4])
        mmin=10
        dmin=1
    else:
        ymin = int(min(row['year'])) 
        if 'month' in row.keys():
            mmin = int(min(row['month']))
        else:
            mmin=1
        if time_agg in ['day','nominal_day']:
            dmin = int(min(row[time_agg])) 
        elif time_agg in ['daily']:
            dmin = int(min(row['day'])) 
        else:
            dmin = 1 # first day of month
        xx,ndays = monthrange(ymin,mmin)     
        if dmin>ndays:
            print('Beware error in allowed dates...')
            # print(row)
            dmin=1
    datemin = pd.Timestamp(f'{ymin}-{mmin}-{dmin}')
    return datemin
def calc_dateminmax_from_cds_form(jfilepath,ecv):
    # Opening JSON file
    f = open(jfilepath)
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
    # print(data)
    df = pd.DataFrame(data)
    # display(df)
    # print(df.keys())
    # print(len(df))
    
    # find records where dates cannot be defined
    if 'sensor_and_algorithm' in df.keys():        
        lst_erase=[]
        for i in range(len(df)):
            if (df['sensor_and_algorithm'][i][0]=='merged_obs4mips'): lst_erase.append(i)
        # now .drop these problematic rows
        for i in lst_erase:
            df=df.drop(lst_erase)
    if ecv == 'Earth Radiation Budget':        
        lst_erase=[]
        for i in range(len(df)):
            if (df['variable'][i][0]=='total_solar_irradiance'): lst_erase.append(i) # this info is read from the dataset itself
        # now .drop these problematic rows
        for i in lst_erase:
            df=df.drop(lst_erase)
    # for i in range(len(df)):
    #     print(df.loc[i])
    #     if ('year' not in df[i]): lst_erase.append(i)

    df['datemax'] = df.apply(compute_datemax,axis=1)
    df['datemin'] = df.apply(compute_datemin,axis=1)
    datemin = df['datemin'].min()
    datemax = df['datemax'].max()
    return datemin,datemax
def datasets_download(datasets_dir):
    c = cdsapi.Client()
    # c.retrieve(
    #     'satellite-ice-sheet-mass-balance',
    #     {
    #         'variable': 'all',
    #         'format': 'zip',
    #     },
    #     datasets_dir+'download.zip')
    # os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
    
    # Antarctica Ice sheet surface elevation change TCDR
    # c.retrieve(
    # 'satellite-ice-sheet-elevation-change',
    # {
    #     'domain': 'antarctica',
    #     'climate_data_record_type': 'tcdr',
    #     'version': '4_0',
    #     'variable': 'all',
    #     'format': 'zip',
    # },
    # datasets_dir+'download.zip')
    # os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
    
    # Antarctica Ice sheet surface elevation change ICDR
    # c.retrieve(
    #     'satellite-ice-sheet-elevation-change',
    # {
    #     'domain': 'antarctica',
    #     'climate_data_record_type': 'icdr',
    #     'version': '3_0',
    #     'variable': 'all',
    #     'format': 'zip',
    # },
    # datasets_dir+'download.zip')
    # os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
    
    
    # Greenland Ice sheet surface elevation change TCDR
    # c.retrieve(
    #     'satellite-ice-sheet-elevation-change',
    # {
    #     'domain': 'greenland',
    #     'climate_data_record_type': 'tcdr',
    #     'version': '4_0',
    #     'variable': 'all',
    #     'format': 'zip',
    # },
    # datasets_dir+'download.zip')
    # os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
    
    # Greenland Ice sheet surface elevation change ICDR
    # c.retrieve(
    #     'satellite-ice-sheet-elevation-change',
    # {
    #     'domain': 'greenland',
    #     'climate_data_record_type': 'icdr',
    #     'version': '4_0',
    #     'variable': 'all',
    #     'format': 'zip',
    # },
    # datasets_dir+'download.zip')
    # os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
    
    # Lake water levels
    
    c.retrieve(
        'satellite-lake-water-level',
    {
        'variable': 'all',
        'version': 'version_4_0',
        'format': 'zip',
        'region': 'northern_africa',
        'lake': 'volta',
    },
    datasets_dir+'download.zip')
    os.system(f'cd {datasets_dir} ; unzip download.zip; rm -rf download.zip ; cd ..')
    
    
    
    
    return
with open('config.yml') as f:
    conf= yaml.safe_load(f)

cds_form_dir=conf['cds_form_dir']
datasets_dir = conf['datasets_dir']

# datasets_download(datasets_dir)


datesbeg = {}
datesend = {}
ecv_dic = {}
for k,ecv in enumerate(conf['ECV']):
    print(ecv)
    entries = conf['ECV'][ecv]['entry']
    print(entries)
    datemin_list = []
    datemax_list = []
    if ecv in ['Earth Radiation Budget']: 
        datemin,datemax = extract_dates_from_TSI()
        datemin_list.append(datemin)
        datemax_list.append(datemax)
    for entry in entries:
        jfilepath=f'{cds_form_dir}{entry}/constraints.json'
        print(jfilepath)
        if jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/satellite-ice-sheet-elevation-change/constraints.json':
            datemin,datemax = extract_dates_icesheets(datasets_dir)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/satellite-ice-sheet-mass-balance/constraints.json':
            datemin,datemax = extract_dates_massbalance(datasets_dir)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/derived-gridded-glacier-mass-change//constraints.json':
            jfile = f'{cds_form_dir}{entry}/generate.yaml'
            datemin,datemax = extract_dates_derived_glaciers(jfile)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/insitu-glaciers-extent/constraints.json':    
            datemin= pd.Timestamp('1990-01-01') # http://www.glims.org/rgi_user_guide/06_dataset_summary.html
            datemax= pd.Timestamp('2010-12-31')
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif jfilepath =='/Users/cxjo/Documents/cds-forms-c3s/satellite-total-column-water-vapour-ocean/constraints.json':
            # temporal aggregation is messed up. does not have the same meaning as other datasets
            # monthly should be yearly
            # 6-hourly should be monthly
            # need to write a special function that accounts for this
            datemin,datemax = extract_dates_wv(jfilepath)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/satellite-lake-water-level/constraints.json':
            datemin,datemax = extract_dates_lake_levels(datasets_dir)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        # elif jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/satellite-land-cover/constraints.json':
        #     datemin_list.append(pd.Timestamp('1992-01-01'))
        #     datemax_list.append(pd.Timestamp('2022-12-31'))
        else:
            # print(jfilepath)
            datemin,datemax = calc_dateminmax_from_cds_form(jfilepath,ecv)
            # print(ecv,datemin_list,datemax_list)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
            # print(ecv,datemin_list,datemax_list)

    # now get the max and min per ECV, accounting for all products
    datemin_list = np.array(datemin_list)
    datemax_list = np.array(datemax_list)
    # datesbeg[ecv] = np.min(datemin_list)
    # datesend[ecv] = np.max(datemax_list)
    ecv_dic[k] = {
        'ECV'     : ecv,
        'DateBeg' : np.min(datemin_list),
        'DateEnd' : np.max(datemax_list),
        'Thematic Hub' : conf['ECV'][ecv]['Thematic_hub']
    }


ModuleNotFoundError: No module named 'matplotlib'

In [21]:



# ecv_pd = pd.DataFrame([conf['ECV'].keys(),datesbeg,datesend],index=['DateBeg','DateEnd']).T
ecv_pd = pd.DataFrame.from_dict(ecv_dic,orient='index').sort_values(['Thematic Hub'])
ecv_pd['DateBeg'] = ecv_pd['DateBeg'].dt.ceil(freq='s')  
ecv_pd['DateEnd'] = ecv_pd['DateEnd'].dt.ceil(freq='s')  
ecv_pd['DateEnd'] = ecv_pd['DateEnd'].apply(lambda dt: dt.strftime("%Y-%m-%d"))
ecv_pd['DateBeg'] = ecv_pd['DateBeg'].apply(lambda dt: dt.strftime("%Y-%m-%d"))

print(ecv_pd.to_markdown())

# fig = px.timeline(ecv_pd, x_start="DateBeg", x_end="DateEnd", y='Product',color='Lot')
fig = px.timeline(ecv_pd, x_start="DateBeg", x_end="DateEnd",y='ECV',color='Thematic Hub')

# fig = px.timeline(datasets_df, x_start="startdate", x_end="enddate", y='ECV')
fig.update_yaxes(autorange="reversed")
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
)
# fig.update_layout(
#     xaxis = dict(
#         dtick = 'Y1',
#         tickformat="%Y",
#     )
# )

xlab = np.arange(1970,2025).astype('int')
xlabtxt = [f'{i}' for i in xlab]


fig.update_xaxes(minor=dict(ticks="inside", showgrid=True))
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = xlab,
        ticktext = xlabtxt
    )
)
fig.update_xaxes(tickangle=-45)
fig.update_layout(
    xaxis = dict(
        tickfont = dict(
            size=10),
        )
    )
fig.update_xaxes(range = ['1970-01-01','2024-12-31'])
# print(fig)
# today_date = pd.Timestamp.today().strftime('%Y%m%d') 
# print(today_date)
# fig.write_image(f'temporal_coverage_by_ECV_{today_date}.pdf')
# fig.write_image(f'temporal_coverage_by_ECV_{today_date}.png')
fig.show()

|    | ECV                      | DateBeg    | DateEnd    | Thematic Hub            |
|---:|:-------------------------|:-----------|:-----------|:------------------------|
|  0 | Aerosols                 | 1995-06-01 | 2023-10-31 | Atmospheric Composition |
|  2 | Greenhouse Gases         | 2002-10-01 | 2021-12-31 | Atmospheric Composition |
| 14 | Ozone                    | 1970-04-01 | 2023-07-31 | Atmospheric Composition |
| 15 | Precipitation            | 1979-01-01 | 2022-06-30 | Atmospheric Physics     |
|  8 | Upper-air Water Vapour   | 1988-01-31 | 2023-02-28 | Atmospheric Physics     |
| 21 | Surface Radiation Budget | 1979-01-01 | 2023-04-30 | Atmospheric Physics     |
|  4 | Earth Radiation Budget   | 1979-01-01 | 2024-03-15 | Atmospheric Physics     |
|  3 | Clouds                   | 1979-01-01 | 2023-04-30 | Atmospheric Physics     |
|  6 | Ice Sheets               | 1992-01-01 | 2023-08-01 | Cryosphere              |
|  7 | Glaciers                 | 1975-04-01 | 2021-09

TypeError: Object of type timedelta is not JSON serializable

datemin: 1992-01-01 00:00:00 datemax: 2023-07-01 00:00:00
